In [4]:
!pip install optuna==4.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 26.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install optuna-integration[sklearn]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 13.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [17]:
!pip install shap==0.47.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 992.3/992.3 kB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 54.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [23]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import os

def load_and_preprocess_data(data_path):
    """Загружает и выполняет базовую предобработку данных SKEMPI с отладочными сообщениями."""
    print(f"\n--- [DEBUG] Загрузка данных из: {data_path} ---")
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Файл {data_path} не найден.")

    # Определяем имена колонок ЗАРАНЕЕ
    expected_column_names = ['#Pdb', 'Mutation(s)_PDB', 'Mutation(s)_cleaned', 'iMutation_Location(s)', 'Hold_out_type', 'Hold_out_proteins', 'Affinity_mut (M)', 'Affinity_mut_parsed', 'Affinity_wt (M)', 'Affinity_wt_parsed', 'Reference', 'Protein 1', 'Protein 2', 'Temperature', 'kon_mut (M^(-1)s^(-1))', 'kon_mut_parsed', 'kon_wt (M^(-1)s^(-1))', 'kon_wt_parsed', 'koff_mut (s^(-1))', 'koff_mut_parsed', 'koff_wt (s^(-1))', 'koff_wt_parsed', 'dH_mut (kcal mol^(-1))', 'dH_wt (kcal mol^(-1))', 'dS_mut (cal mol^(-1) K^(-1))', 'dS_wt (cal mol^(-1) K^(-1))', 'Notes', 'Method', 'SKEMPI version']

    # Пытаемся прочитать с заголовком из первой строки
    try:
        print("[DEBUG] Попытка чтения с header=0, sep=';'...")
        df = pd.read_csv(data_path, sep=';', header=0, low_memory=False) # Added low_memory=False
        print(f"[DEBUG] Размер после загрузки (header=0): {df.shape}")
        # Если колонки не прочитались как надо, присваиваем стандартные
        if len(df.columns) != len(expected_column_names):
             print(f"[DEBUG] ВНИМАНИЕ: Кол-во колонок ({len(df.columns)}) не совпадает с ожидаемым ({len(expected_column_names)}). Переименовываем...")
             if len(df.columns) < len(expected_column_names):
                 df.columns = expected_column_names[:len(df.columns)]
             else: # Больше колонок, чем ожидалось
                 df.columns = expected_column_names + [f'extra_{i}' for i in range(len(df.columns) - len(expected_column_names))]
        else:
             # На всякий случай переименуем, если первая строка была не совсем заголовком
             df.columns = expected_column_names
             print("[DEBUG] Колонки переименованы в стандартные.")

    except Exception as e:
        print(f"[DEBUG] Ошибка при чтении CSV: {e}")
        # Попробуем без header если первая попытка не удалась или колонки не совпали
        try:
            print("[DEBUG] Попытка чтения без header, с присвоением имен...")
            df = pd.read_csv(data_path, sep=';', header=None, names=expected_column_names, low_memory=False)
            print(f"[DEBUG] Размер после загрузки (header=None): {df.shape}")
            # Удалим первую строку, если она была заголовком
            if df.iloc[0]['#Pdb'] == '#Pdb': # Проверяем, похожа ли первая строка на заголовок
                df = df.iloc[1:].reset_index(drop=True)
                print("[DEBUG] Первая строка, похожая на заголовок, удалена.")
        except Exception as e2:
             print(f"[DEBUG] Вторая ошибка при чтении CSV: {e2}")
             return pd.DataFrame(), [], []


    if df.empty:
        print("[DEBUG] ОШИБКА: DataFrame пуст сразу после загрузки.")
        return pd.DataFrame(), [], []

    # --- Фильтрация Мутаций (ОСТОРОЖНО) ---
    if 'Mutation(s)_cleaned' not in df.columns:
        print("[DEBUG] ОШИБКА: Столбец 'Mutation(s)_cleaned' не найден после загрузки/переименования.")
        return pd.DataFrame(), [], []

    original_rows = df.shape[0]
    print(f"[DEBUG] Начальный размер перед фильтрацией мутаций: {original_rows}")

    # 1. Убираем NaN в колонке мутаций
    df = df.dropna(subset=['Mutation(s)_cleaned'])
    print(f"[DEBUG] Размер после dropna('Mutation(s)_cleaned'): {df.shape} (удалено {original_rows - df.shape[0]})")
    if df.empty: return pd.DataFrame(), [], []

    # 2. Извлекаем WT и MUT аминокислоты (первый и последний символ)
    # Преобразуем в строку на всякий случай
    df['WT_AA'] = df['Mutation(s)_cleaned'].astype(str).str[0]
    df['MUT_AA'] = df['Mutation(s)_cleaned'].astype(str).str[-1]
    print("[DEBUG] WT_AA и MUT_AA извлечены.")

    # 3. Проверяем, что WT и MUT - валидные аминокислоты (буквы из словаря)
    aa_properties = { 'A': {'volume': 88.6, 'hydrophobicity': 1.8}, 'R': {'volume': 173.4, 'hydrophobicity': -4.5}, 'N': {'volume': 114.1, 'hydrophobicity': -3.5}, 'D': {'volume': 111.1, 'hydrophobicity': -3.5}, 'C': {'volume': 108.5, 'hydrophobicity': 2.5}, 'Q': {'volume': 143.8, 'hydrophobicity': -3.5}, 'E': {'volume': 138.4, 'hydrophobicity': -3.5}, 'G': {'volume': 60.1, 'hydrophobicity': -0.4}, 'H': {'volume': 153.2, 'hydrophobicity': -3.2}, 'I': {'volume': 166.7, 'hydrophobicity': 4.5}, 'L': {'volume': 166.7, 'hydrophobicity': 3.8}, 'K': {'volume': 168.6, 'hydrophobicity': -3.9}, 'M': {'volume': 162.9, 'hydrophobicity': 1.9}, 'F': {'volume': 189.9, 'hydrophobicity': 2.8}, 'P': {'volume': 112.7, 'hydrophobicity': -1.6}, 'S': {'volume': 89.0, 'hydrophobicity': -0.8}, 'T': {'volume': 116.1, 'hydrophobicity': -0.7}, 'W': {'volume': 227.8, 'hydrophobicity': -0.9}, 'Y': {'volume': 193.6, 'hydrophobicity': -1.3}, 'V': {'volume': 140.0, 'hydrophobicity': 4.2}}
    valid_aa = set(aa_properties.keys())
    original_rows_aa_filter = df.shape[0]
    df = df[df['WT_AA'].isin(valid_aa) & df['MUT_AA'].isin(valid_aa)]
    print(f"[DEBUG] Размер после фильтрации невалидных WT/MUT AA: {df.shape} (удалено {original_rows_aa_filter - df.shape[0]})")
    if df.empty:
         print("[DEBUG] ОШИБКА: DataFrame пуст после фильтрации невалидных WT/MUT AA.")
         return pd.DataFrame(), [], []

    # 4. Убираем мутации A->A, C->C и т.д.
    original_rows_self_mut = df.shape[0]
    df = df[df['WT_AA'] != df['MUT_AA']]
    print(f"[DEBUG] Размер после удаления мутаций A->A: {df.shape} (удалено {original_rows_self_mut - df.shape[0]})")
    if df.empty:
        print("[DEBUG] ОШИБКА: DataFrame пуст после удаления мутаций A->A.")
        return pd.DataFrame(), [], []

    # --- Остальная обработка (как в v2) ---
    # Обработка температуры
    if 'Temperature' in df.columns:
        df['Temperature'] = df['Temperature'].astype(str).str.replace(r'\(assumed\)', '', regex=True)
        # Handle ranges like '298-310' by taking the average or first value
        def parse_temp(temp_str):
            if '-' in temp_str:
                try:
                    low, high = map(float, temp_str.split('-'))
                    return (low + high) / 2
                except:
                    return np.nan
            else:
                return pd.to_numeric(temp_str, errors='coerce')

        df['Temperature'] = df['Temperature'].apply(parse_temp)
        nan_before = df['Temperature'].isna().sum()
        if df['Temperature'].notna().any() and nan_before < len(df): # Check if there are non-NaNs to impute from
            try:
                 imputer = KNNImputer(n_neighbors=5)
                 df['Temperature'] = imputer.fit_transform(df[['Temperature']])
            except ValueError as e:
                 print(f"[DEBUG] Warning: KNNImputer failed for Temperature ({e}). Filling NaN with mean.")
                 mean_temp = df['Temperature'].mean()
                 df['Temperature'] = df['Temperature'].fillna(mean_temp if not pd.isna(mean_temp) else 298.0)
        else: # All NaNs or no NaNs
            mean_temp = df['Temperature'].mean()
            df['Temperature'] = df['Temperature'].fillna(mean_temp if not pd.isna(mean_temp) else 298.0) # Fill remaining NaNs (if any) or if all were NaNs

        print(f"[DEBUG] Температура обработана. NaN до (после парсинга): {nan_before}, NaN после: {df['Temperature'].isna().sum()}")
    else:
        print("[DEBUG] ВНИМАНИЕ: Столбец 'Temperature' не найден. Используем T=298K.")
        df['Temperature'] = 298.0

    # Вычисление ddG
    if 'Affinity_mut_parsed' in df.columns and 'Affinity_wt_parsed' in df.columns:
        R = 1.987e-3
        df['Kd_mut'] = pd.to_numeric(df['Affinity_mut_parsed'], errors='coerce')
        df['Kd_wt'] = pd.to_numeric(df['Affinity_wt_parsed'], errors='coerce')
        mask = (df['Kd_mut'].notnull()) & (df['Kd_wt'].notnull()) & \
               (df['Kd_wt'] > 0) & (df['Kd_mut'] > 0) & \
               (df['Temperature'].notnull()) & (df['Temperature'] > 0)
        df['ddG'] = np.nan
        df.loc[mask, 'ddG'] = R * df.loc[mask, 'Temperature'] * np.log(df.loc[mask, 'Kd_mut'] / df.loc[mask, 'Kd_wt']) # Note: original formula had -R, SKEMPI ddG = RT*ln(Kd_mut/Kd_wt)
        print(f"[DEBUG] Строк с валидными Kd/T для ddG: {mask.sum()}")
        original_rows = df.shape[0]
        df = df.dropna(subset=['ddG'])
        print(f"[DEBUG] Размер после dropna(ddG): {df.shape} (удалено {original_rows - df.shape[0]})")
        if df.empty: return pd.DataFrame(), [], []
        original_rows = df.shape[0]
        df = df[df['ddG'] != 0]
        print(f"[DEBUG] Размер после удаления ddG==0: {df.shape} (удалено {original_rows - df.shape[0]})")
        if df.empty: return pd.DataFrame(), [], []
        # Sign definition: positive ddG -> destabilizing (weaker binding Kd_mut > Kd_wt) -> Class 0
        #                  negative ddG -> stabilizing (stronger binding Kd_mut < Kd_wt) -> Class 1
        df['sign_ddG'] = (df['ddG'] < 0).astype(int)
        print(f"[DEBUG] Распределение sign_ddG (1=стабилизация, 0=дестабилизация):\n{df['sign_ddG'].value_counts(normalize=True)}")

    else:
         print("[DEBUG] ОШИБКА: Столбцы аффинности не найдены.")
         return pd.DataFrame(), [], []

    # Инженерия признаков
    df['Position'] = df['Mutation(s)_cleaned'].astype(str).str.extract(r'(\d+)', expand=False).astype(float)
    if 'iMutation_Location(s)' in df.columns:
        df['Location'] = df['iMutation_Location(s)'].fillna('Unknown')
        # Map detailed locations to broader categories if desired (like in previous script)
        location_map = {'COR': 'COR', 'RIM': 'SUR', 'SUR': 'SUR', 'SUP': 'INT', 'INT': 'INT'}
        df['Location_General'] = df['Location'].map(location_map).fillna('Other') # Use a general category
        print(f"[DEBUG] Location_General categories:\n{df['Location_General'].value_counts()}")
    else:
        print("[DEBUG] ВНИМАНИЕ: столбец 'iMutation_Location(s)' не найден.")
        df['Location'] = 'Unknown'
        df['Location_General'] = 'Unknown'


    # Физико-химические свойства
    for prop in ['volume', 'hydrophobicity']:
        prop_map = {aa: props[prop] for aa, props in aa_properties.items()}
        wt_props = df['WT_AA'].map(prop_map)
        mut_props = df['MUT_AA'].map(prop_map)
        df[f'delta_{prop}'] = mut_props - wt_props
        df[f'WT_{prop}'] = wt_props
        df[f'MUT_{prop}'] = mut_props
    print("[DEBUG] Физ.-хим. признаки рассчитаны.")

    # Use Location_General for features
    categorical_features = ['WT_AA', 'MUT_AA', 'Location_General']
    numeric_features = ['Position', 'delta_volume', 'delta_hydrophobicity',
                       'WT_volume', 'WT_hydrophobicity', 'MUT_volume', 'MUT_hydrophobicity']
    final_feature_cols = categorical_features + numeric_features
    target_col = 'sign_ddG'

    missing_features = [col for col in final_feature_cols if col not in df.columns]
    if missing_features:
        print(f"[DEBUG] ОШИБКА: Отсутствуют колонки признаков: {missing_features}")
        return pd.DataFrame(), [], []
    if target_col not in df.columns:
         print(f"[DEBUG] ОШИБКА: Отсутствует целевая колонка '{target_col}'.")
         return pd.DataFrame(), [], []

    # Drop NaNs from final feature columns + target BEFORE returning
    original_rows = df.shape[0]
    df_clean = df[final_feature_cols + [target_col]].copy() # Select only needed cols before dropna
    df_clean = df_clean.dropna(subset=final_feature_cols + [target_col])
    print(f"[DEBUG] Размер после dropna(признаки+цель): {df_clean.shape} (удалено {original_rows - df_clean.shape[0]})")

    # Ensure numeric types after potential NaNs
    for col in numeric_features:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    original_rows = df_clean.shape[0]
    df_clean = df_clean.dropna(subset=numeric_features) # Drop again if coerce created NaNs
    print(f"[DEBUG] Размер после dropna(num_features post-convert): {df_clean.shape} (удалено {original_rows - df_clean.shape[0]})")

    if df_clean.empty:
        print("[DEBUG] ОШИБКА: DataFrame пуст перед возвратом.")
    else:
        print(f"--- [DEBUG] Данные успешно обработаны. Финальный размер: {df_clean.shape} ---")

    # Return the cleaned dataframe and the correct feature lists
    return df_clean, categorical_features, numeric_features

In [ ]:
# ========= 1. Импорты и Настройки =========
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from joblib import Memory
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (accuracy_score, balanced_accuracy_score, cohen_kappa_score,
                             f1_score, matthews_corrcoef, log_loss, roc_auc_score,
                             average_precision_score, make_scorer, confusion_matrix)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline as SklearnPipeline
from sklearn.base import clone
from sklearn.impute import KNNImputer
from scipy.sparse import issparse
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline as ImbPipeline
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntDistribution, FloatDistribution, CategoricalDistribution
import shap
import json

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

# Конфигурация
DATA_PATH = 'skempi_v2.csv'
LOCATION_TO_ANALYZE = 'Other'
TEST_SIZE = 0.25
RANDOM_STATE = 42
CV_FOLDS = 5
N_FEATURES_TO_SELECT = 30

# ========= 2. Расширенная Предобработка =========
def enhanced_preprocessing(df):
    """Добавление структурных и эволюционных признаков"""
    # Добавление структурных признаков (гипотетических значений)
    df['delta_rsa'] = np.random.normal(0, 1, len(df))  # ΔSASA
    df['conservation_score'] = np.random.normal(0.5, 0.2, len(df))  # Консервативность
    
    # Расширенные физико-химические свойства
    aa_properties = {
        'A': {'volume': 88.6, 'hydrophobicity': 1.8, 'polarity': 0},
        'R': {'volume': 173.4, 'hydrophobicity': -4.5, 'polarity': 1},
        # ...остальные аминокислоты с дополнительными свойствами
    }
    
    # Добавление новых признаков
    df['charge_diff'] = df['MUT_AA'].map(lambda x: 1 if x in 'KR' else (-1 if x in 'DE' else 0)) - \
                        df['WT_AA'].map(lambda x: 1 if x in 'KR' else (-1 if x in 'DE' else 0))
    
    return df

# ========= 3. Загрузка и Подготовка Данных =========
df_processed, categorical_features, numeric_features = load_and_preprocess_data(DATA_PATH)
df_processed = enhanced_preprocessing(df_processed)

# Фильтрация по локации
if LOCATION_TO_ANALYZE == 'Other':
    df_loc = df_processed[df_processed['Location_General'] != 'COR']
elif LOCATION_TO_ANALYZE == 'COR':
    df_loc = df_processed[df_processed['Location_General'] == 'COR']
else:
    df_loc = df_processed.copy()

# Добавление новых признаков в список
numeric_features.extend(['delta_rsa', 'conservation_score', 'charge_diff'])

X = df_loc[categorical_features + numeric_features]
y = df_loc['sign_ddG']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# ========= 4. Расширенный Препроцессор =========
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'), categorical_features),
        ('num', StandardScaler(), numeric_features)
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
)

# ========= 5. Лучшие Модели с Гиперпараметрической Оптимизацией =========
# Базовые модели для ансамбля
rf_params = {
    'n_estimators': IntDistribution(100, 500),
    'max_depth': IntDistribution(3, 20),
    'min_samples_split': IntDistribution(2, 11),
    'class_weight': CategoricalDistribution(['balanced', None])
}

gb_params = {
    'n_estimators': IntDistribution(100, 500),
    'learning_rate': FloatDistribution(0.01, 0.3),
    'max_depth': IntDistribution(3, 15),
    'subsample': FloatDistribution(0.6, 1.0)
}

svc_params = {
    'C': FloatDistribution(0.1, 100),
    'kernel': CategoricalDistribution(['rbf', 'poly']),
    'gamma': CategoricalDistribution(['scale', 'auto'])
}

# Создание оптимизаторов
opt_rf = OptunaSearchCV(
    estimator=RandomForestClassifier(random_state=RANDOM_STATE),
    param_distributions=rf_params,
    cv=StratifiedKFold(n_splits=CV_FOLDS),
    n_trials=30,
    scoring='f1',
    n_jobs=-1
)

opt_gb = OptunaSearchCV(
    estimator=GradientBoostingClassifier(random_state=RANDOM_STATE),
    param_distributions=gb_params,
    cv=StratifiedKFold(n_splits=CV_FOLDS),
    n_trials=30,
    scoring='f1',
    n_jobs=-1
)

# ========= 6. Ансамблевая Модель =========
stacking_model = StackingClassifier(
    estimators=[
        ('rf', opt_rf),
        ('gb', opt_gb),
        ('lr', LogisticRegression(max_iter=1000))
    ],
    final_estimator=LogisticRegression(),
    cv=StratifiedKFold(n_splits=CV_FOLDS)
)

# ========= 7. Пайплайн с Ресэмплингом =========
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', SelectKBest(score_func=f_classif, k=N_FEATURES_TO_SELECT)),
    ('resampler', SMOTE(random_state=RANDOM_STATE)),
    ('classifier', stacking_model)
])

# ========= 8. Обучение и Оценка =========
# Обучение
pipeline.fit(X_train, y_train)

# Предсказание
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Расчет метрик
metrics = calculate_metrics(y_test, y_pred, y_proba)

# ========= 9. Интерпретация =========
# Получение важных признаков
explainer = shap.Explainer(pipeline.named_steps['classifier'])
shap_values = explainer.shap_values(pipeline.named_steps['preprocessor'].transform(X_test))

# Визуализация
plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values, pipeline.named_steps['preprocessor'].transform(X_test))
plt.tight_layout()
plt.savefig('shap_summary.png')

# ========= 10. Сохранение Результатов =========
results = {
    'best_params': {
        'rf': opt_rf.best_params_,
        'gb': opt_gb.best_params_
    },
    'metrics': metrics,
    'feature_importance': dict(zip(
        pipeline.named_steps['preprocessor'].get_feature_names_out(),
        pipeline.named_steps['classifier'].feature_importances_
    ))
}

with open('enhanced_model_results.json', 'w') as f:
    json.dump(results, f, indent=4)

# Сохранение модели
import joblib
joblib.dump(pipeline, 'enhanced_model.joblib')

print("Расширенная модель успешно обучена и сохранена!")


--- [DEBUG] Загрузка данных из: skempi_v2.csv ---
[DEBUG] Попытка чтения с header=0, sep=';'...
[DEBUG] Размер после загрузки (header=0): (7085, 29)
[DEBUG] Колонки переименованы в стандартные.
[DEBUG] Начальный размер перед фильтрацией мутаций: 7085
[DEBUG] Размер после dropna('Mutation(s)_cleaned'): (7085, 29) (удалено 0)
[DEBUG] WT_AA и MUT_AA извлечены.
[DEBUG] Размер после фильтрации невалидных WT/MUT AA: (7085, 31) (удалено 0)
[DEBUG] Размер после удаления мутаций A->A: (6992, 31) (удалено 93)
[I 2025-05-01 00:43:18,850] A new study created in memory with name: no-name-cb2983c6-b947-47df-b783-b00fcc8c59ed
[DEBUG] Температура обработана. NaN до (после парсинга): 4, NaN после: 0
[DEBUG] Строк с валидными Kd/T для ddG: 6719
[DEBUG] Размер после dropna(ddG): (6719, 34) (удалено 273)
[DEBUG] Размер после удаления ddG==0: (6531, 34) (удалено 188)
[DEBUG] Распределение sign_ddG (1=стабилизация, 0=дестабилизация):
sign_ddG
0    0.78181
1    0.21819
Name: proportion, dtype: float64
[DEBU

In [ ]:
# ========= 1. Импорты и Настройки =========
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from joblib import Memory
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (accuracy_score, balanced_accuracy_score, cohen_kappa_score,
                             f1_score, matthews_corrcoef, log_loss, roc_auc_score,
                             average_precision_score, make_scorer, confusion_matrix)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline as SklearnPipeline
from sklearn.base import clone
from sklearn.impute import KNNImputer
from scipy.sparse import issparse
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline as ImbPipeline
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntDistribution, FloatDistribution, CategoricalDistribution
import shap
import json

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

# Конфигурация
DATA_PATH = 'skempi_v2.csv'
LOCATION_TO_ANALYZE = 'Other'
TEST_SIZE = 0.25
RANDOM_STATE = 42
CV_FOLDS = 5
N_FEATURES_TO_SELECT = 30

# ========= 2. Расширенная Предобработка =========
def enhanced_preprocessing(df):
    """Добавление структурных и эволюционных признаков"""
    # Добавление структурных признаков (гипотетических значений)
    df['delta_rsa'] = np.random.normal(0, 1, len(df))  # ΔSASA
    df['conservation_score'] = np.random.normal(0.5, 0.2, len(df))  # Консервативность
    
    # Расширенные физико-химические свойства
    aa_properties = {
        'A': {'volume': 88.6, 'hydrophobicity': 1.8, 'polarity': 0},
        'R': {'volume': 173.4, 'hydrophobicity': -4.5, 'polarity': 1},
        'N': {'volume': 114.1, 'hydrophobicity': -3.5, 'polarity': 1},
        'D': {'volume': 111.1, 'hydrophobicity': -3.5, 'polarity': 1},
        'C': {'volume': 108.5, 'hydrophobicity': 2.5, 'polarity': 0},
        'Q': {'volume': 143.8, 'hydrophobicity': -3.5, 'polarity': 1},
        'E': {'volume': 138.4, 'hydrophobicity': -3.5, 'polarity': 1},
        'G': {'volume': 60.1, 'hydrophobicity': -0.4, 'polarity': 0},
        'H': {'volume': 153.2, 'hydrophobicity': -3.2, 'polarity': 1},
        'I': {'volume': 166.7, 'hydrophobicity': 4.5, 'polarity': 0},
        'L': {'volume': 166.7, 'hydrophobicity': 3.8, 'polarity': 0},
        'K': {'volume': 168.6, 'hydrophobicity': -3.9, 'polarity': 1},
        'M': {'volume': 162.9, 'hydrophobicity': 1.9, 'polarity': 0},
        'F': {'volume': 189.9, 'hydrophobicity': 2.8, 'polarity': 0},
        'P': {'volume': 112.7, 'hydrophobicity': -1.6, 'polarity': 0},
        'S': {'volume': 89.0, 'hydrophobicity': -0.8, 'polarity': 1},
        'T': {'volume': 116.1, 'hydrophobicity': -0.7, 'polarity': 1},
        'W': {'volume': 227.8, 'hydrophobicity': -0.9, 'polarity': 0},
        'Y': {'volume': 193.6, 'hydrophobicity': -1.3, 'polarity': 1},
        'V': {'volume': 140.0, 'hydrophobicity': 4.2, 'polarity': 0}
    }
    
    # Добавление новых признаков
    df['charge_diff'] = df['MUT_AA'].map(lambda x: 1 if x in 'KR' else (-1 if x in 'DE' else 0)) - \
                        df['WT_AA'].map(lambda x: 1 if x in 'KR' else (-1 if x in 'DE' else 0))
    
    return df

# ========= 3. Загрузка и Подготовка Данных =========
def load_and_preprocess_data(data_path):
    """Загружает и выполняет базовую предобработку данных SKEMPI с отладочными сообщениями."""
    print(f"\n--- [DEBUG] Загрузка данных из: {data_path} ---")
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Файл {data_path} не найден.")
    
    expected_column_names = ['#Pdb', 'Mutation(s)_PDB', 'Mutation(s)_cleaned', 'iMutation_Location(s)', 'Hold_out_type', 'Hold_out_proteins', 'Affinity_mut (M)', 'Affinity_mut_parsed', 'Affinity_wt (M)', 'Affinity_wt_parsed', 'Reference', 'Protein 1', 'Protein 2', 'Temperature', 'kon_mut (M^(-1)s^(-1))', 'kon_mut_parsed', 'kon_wt (M^(-1)s^(-1))', 'kon_wt_parsed', 'koff_mut (s^(-1))', 'koff_mut_parsed', 'koff_wt (s^(-1))', 'koff_wt_parsed', 'dH_mut (kcal mol^(-1))', 'dH_wt (kcal mol^(-1))', 'dS_mut (cal mol^(-1) K^(-1))', 'dS_wt (cal mol^(-1) K^(-1))', 'Notes', 'Method', 'SKEMPI version']
    
    try:
        print("[DEBUG] Попытка чтения с header=0, sep=';'...")
        df = pd.read_csv(data_path, sep=';', header=0, low_memory=False)
        print(f"[DEBUG] Размер после загрузки (header=0): {df.shape}")
        
        if len(df.columns) != len(expected_column_names):
            print(f"[DEBUG] ВНИМАНИЕ: Кол-во колонок ({len(df.columns)}) не совпадает с ожидаемым ({len(expected_column_names)}). Переименовываем...")
            if len(df.columns) < len(expected_column_names):
                df.columns = expected_column_names[:len(df.columns)]
            else:
                df.columns = expected_column_names + [f'extra_{i}' for i in range(len(df.columns) - len(expected_column_names))]
        else:
            df.columns = expected_column_names
            print("[DEBUG] Колонки переименованы в стандартные.")
    except Exception as e:
        print(f"[DEBUG] Ошибка при чтении CSV: {e}")
        try:
            print("[DEBUG] Попытка чтения без header, с присвоением имен...")
            df = pd.read_csv(data_path, sep=';', header=None, names=expected_column_names, low_memory=False)
            print(f"[DEBUG] Размер после загрузки (header=None): {df.shape}")
            if df.iloc[0]['#Pdb'] == '#Pdb':
                df = df.iloc[1:].reset_index(drop=True)
                print("[DEBUG] Первая строка, похожая на заголовок, удалена.")
        except Exception as e2:
            print(f"[DEBUG] Вторая ошибка при чтении CSV: {e2}")
            return pd.DataFrame(), [], []
    
    if df.empty:
        print("[DEBUG] ОШИБКА: DataFrame пуст сразу после загрузки.")
        return pd.DataFrame(), [], []
    
    if 'Mutation(s)_cleaned' not in df.columns:
        print("[DEBUG] ОШИБКА: Столбец 'Mutation(s)_cleaned' не найден после загрузки/переименования.")
        return pd.DataFrame(), [], []
    
    original_rows = df.shape[0]
    print(f"[DEBUG] Начальный размер перед фильтрацией мутаций: {original_rows}")
    
    df = df.dropna(subset=['Mutation(s)_cleaned'])
    print(f"[DEBUG] Размер после dropna('Mutation(s)_cleaned'): {df.shape} (удалено {original_rows - df.shape[0]})")
    
    if df.empty: return pd.DataFrame(), [], []
    
    df['WT_AA'] = df['Mutation(s)_cleaned'].astype(str).str[0]
    df['MUT_AA'] = df['Mutation(s)_cleaned'].astype(str).str[-1]
    print("[DEBUG] WT_AA и MUT_AA извлечены.")
    
    aa_properties = { 'A': {'volume': 88.6, 'hydrophobicity': 1.8}, 'R': {'volume': 173.4, 'hydrophobicity': -4.5}, 'N': {'volume': 114.1, 'hydrophobicity': -3.5}, 'D': {'volume': 111.1, 'hydrophobicity': -3.5}, 'C': {'volume': 108.5, 'hydrophobicity': 2.5}, 'Q': {'volume': 143.8, 'hydrophobicity': -3.5}, 'E': {'volume': 138.4, 'hydrophobicity': -3.5}, 'G': {'volume': 60.1, 'hydrophobicity': -0.4}, 'H': {'volume': 153.2, 'hydrophobicity': -3.2}, 'I': {'volume': 166.7, 'hydrophobicity': 4.5}, 'L': {'volume': 166.7, 'hydrophobicity': 3.8}, 'K': {'volume': 168.6, 'hydrophobicity': -3.9}, 'M': {'volume': 162.9, 'hydrophobicity': 1.9}, 'F': {'volume': 189.9, 'hydrophobicity': 2.8}, 'P': {'volume': 112.7, 'hydrophobicity': -1.6}, 'S': {'volume': 89.0, 'hydrophobicity': -0.8}, 'T': {'volume': 116.1, 'hydrophobicity': -0.7}, 'W': {'volume': 227.8, 'hydrophobicity': -0.9}, 'Y': {'volume': 193.6, 'hydrophobicity': -1.3}, 'V': {'volume': 140.0, 'hydrophobicity': 4.2}}
    
    valid_aa = set(aa_properties.keys())
    original_rows_aa_filter = df.shape[0]
    df = df[df['WT_AA'].isin(valid_aa) & df['MUT_AA'].isin(valid_aa)]
    print(f"[DEBUG] Размер после фильтрации невалидных WT/MUT AA: {df.shape} (удалено {original_rows_aa_filter - df.shape[0]})")
    
    if df.empty:
        print("[DEBUG] ОШИБКА: DataFrame пуст после фильтрации невалидных WT/MUT AA.")
        return pd.DataFrame(), [], []
    
    original_rows_self_mut = df.shape[0]
    df = df[df['WT_AA'] != df['MUT_AA']]
    print(f"[DEBUG] Размер после удаления мутаций A->A: {df.shape} (удалено {original_rows_self_mut - df.shape[0]})")
    
    if df.empty:
        print("[DEBUG] ОШИБКА: DataFrame пуст после удаления мутаций A->A.")
        return pd.DataFrame(), [], []
    
    if 'Temperature' in df.columns:
        df['Temperature'] = df['Temperature'].astype(str).str.replace(r'$assumed$', '', regex=True)
        
        def parse_temp(temp_str):
            if '-' in temp_str:
                try:
                    low, high = map(float, temp_str.split('-'))
                    return (low + high) / 2
                except:
                    return np.nan
            else:
                return pd.to_numeric(temp_str, errors='coerce')
        
        df['Temperature'] = df['Temperature'].apply(parse_temp)
        nan_before = df['Temperature'].isna().sum()
        
        if df['Temperature'].notna().any() and nan_before < len(df):
            try:
                imputer = KNNImputer(n_neighbors=5)
                df['Temperature'] = imputer.fit_transform(df[['Temperature']])
            except ValueError as e:
                print(f"[DEBUG] Warning: KNNImputer failed for Temperature ({e}). Filling NaN with mean.")
                mean_temp = df['Temperature'].mean()
                df['Temperature'] = df['Temperature'].fillna(mean_temp if not pd.isna(mean_temp) else 298.0)
        else:
            mean_temp = df['Temperature'].mean()
            df['Temperature'] = df['Temperature'].fillna(mean_temp if not pd.isna(mean_temp) else 298.0)
        
        print(f"[DEBUG] Температура обработана. NaN до (после парсинга): {nan_before}, NaN после: {df['Temperature'].isna().sum()}")
    else:
        print("[DEBUG] ВНИМАНИЕ: Столбец 'Temperature' не найден. Используем T=298K.")
        df['Temperature'] = 298.0
    
    if 'Affinity_mut_parsed' in df.columns and 'Affinity_wt_parsed' in df.columns:
        R = 1.987e-3
        df['Kd_mut'] = pd.to_numeric(df['Affinity_mut_parsed'], errors='coerce')
        df['Kd_wt'] = pd.to_numeric(df['Affinity_wt_parsed'], errors='coerce')
        
        mask = (df['Kd_mut'].notnull()) & (df['Kd_wt'].notnull()) & \
               (df['Kd_wt'] > 0) & (df['Kd_mut'] > 0) & \
               (df['Temperature'].notnull()) & (df['Temperature'] > 0)
        
        df['ddG'] = np.nan
        df.loc[mask, 'ddG'] = R * df.loc[mask, 'Temperature'] * np.log(df.loc[mask, 'Kd_mut'] / df.loc[mask, 'Kd_wt'])
        
        print(f"[DEBUG] Строк с валидными Kd/T для ddG: {mask.sum()}")
        original_rows = df.shape[0]
        df = df.dropna(subset=['ddG'])
        print(f"[DEBUG] Размер после dropna(ddG): {df.shape} (удалено {original_rows - df.shape[0]})")
        
        if df.empty: return pd.DataFrame(), [], []
        
        original_rows = df.shape[0]
        df = df[df['ddG'] != 0]
        print(f"[DEBUG] Размер после удаления ddG==0: {df.shape} (удалено {original_rows - df.shape[0]})")
        
        if df.empty: return pd.DataFrame(), [], []
        
        df['sign_ddG'] = (df['ddG'] < 0).astype(int)
        print(f"[DEBUG] Распределение sign_ddG (1=стабилизация, 0=дестабилизация):\n{df['sign_ddG'].value_counts(normalize=True)}")
    else:
        print("[DEBUG] ОШИБКА: Столбцы аффинности не найдены.")
        return pd.DataFrame(), [], []
    
    df['Position'] = df['Mutation(s)_cleaned'].astype(str).str.extract(r'(\d+)', expand=False).astype(float)
    
    if 'iMutation_Location(s)' in df.columns:
        df['Location'] = df['iMutation_Location(s)'].fillna('Unknown')
        location_map = {'COR': 'COR', 'RIM': 'SUR', 'SUR': 'SUR', 'SUP': 'INT', 'INT': 'INT'}
        df['Location_General'] = df['Location'].map(location_map).fillna('Other')
        print(f"[DEBUG] Location_General categories:\n{df['Location_General'].value_counts()}")
    else:
        print("[DEBUG] ВНИМАНИЕ: столбец 'iMutation_Location(s)' не найден.")
        df['Location'] = 'Unknown'
        df['Location_General'] = 'Unknown'
    
    for prop in ['volume', 'hydrophobicity']:
        prop_map = {aa: props[prop] for aa, props in aa_properties.items()}
        wt_props = df['WT_AA'].map(prop_map)
        mut_props = df['MUT_AA'].map(prop_map)
        df[f'delta_{prop}'] = mut_props - wt_props
        df[f'WT_{prop}'] = wt_props
        df[f'MUT_{prop}'] = mut_props
    
    print("[DEBUG] Физ.-хим. признаки рассчитаны.")
    
    categorical_features = ['WT_AA', 'MUT_AA', 'Location_General']
    numeric_features = ['Position', 'delta_volume', 'delta_hydrophobicity',
                       'WT_volume', 'WT_hydrophobicity', 'MUT_volume', 'MUT_hydrophobicity']
    
    final_feature_cols = categorical_features + numeric_features
    target_col = 'sign_ddG'
    
    missing_features = [col for col in final_feature_cols if col not in df.columns]
    if missing_features:
        print(f"[DEBUG] ОШИБКА: Отсутствуют колонки признаков: {missing_features}")
        return pd.DataFrame(), [], []
    
    if target_col not in df.columns:
        print(f"[DEBUG] ОШИБКА: Отсутствует целевая колонка '{target_col}'.")
        return pd.DataFrame(), [], []
    
    original_rows = df.shape[0]
    df_clean = df[final_feature_cols + [target_col]].copy()
    df_clean = df_clean.dropna(subset=final_feature_cols + [target_col])
    print(f"[DEBUG] Размер после dropna(признаки+цель): {df_clean.shape} (удалено {original_rows - df_clean.shape[0]})")
    
    for col in numeric_features:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    original_rows = df_clean.shape[0]
    df_clean = df_clean.dropna(subset=numeric_features)
    print(f"[DEBUG] Размер после dropna(num_features post-convert): {df_clean.shape} (удалено {original_rows - df_clean.shape[0]})")
    
    if df_clean.empty:
        print("[DEBUG] ОШИБКА: DataFrame пуст перед возвратом.")
    else:
        print(f"--- [DEBUG] Данные успешно обработаны. Финальный размер: {df_clean.shape} ---")
    
    return df_clean, categorical_features, numeric_features

# ========= 4. Функции Оценки и Подбора Порога =========
def calculate_metrics(y_true, y_pred, y_proba):
    """Рассчитывает набор метрик для бинарной классификации."""
    if y_proba.ndim > 1 and y_proba.shape[1] == 2:
        y_proba = y_proba[:, 1]
    elif y_proba.ndim > 1 and y_proba.shape[1] > 2:
        raise ValueError("y_proba должен содержать вероятности для бинарной классификации")
    
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    
    unique_preds = np.unique(y_pred)
    pos_label = 1
    
    f1 = f1_score(y_true, y_pred, pos_label=pos_label, zero_division=0)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    avg_prec = 0.0
    if pos_label in y_true:
        try:
            avg_prec = average_precision_score(y_true, y_proba, pos_label=pos_label)
        except ValueError as e:
            print(f"  Warning calculating PR AUC: {e}. Setting to 0.0")
            avg_prec = 0.0
    else:
        print(f"  Warning: Positive label {pos_label} not found in y_true for PR AUC calculation.")
        avg_prec = np.nan
    
    eps = 1e-15
    y_proba_clipped = np.clip(y_proba, eps, 1 - eps)
    logloss = log_loss(y_true, y_proba_clipped)
    
    roc_auc = 0.0
    if len(np.unique(y_true)) > 1:
        try:
            roc_auc = roc_auc_score(y_true, y_proba)
        except ValueError as e:
            print(f"  Warning calculating ROC AUC: {e}. Setting to 0.0")
            roc_auc = 0.0
    else:
        print("  Warning: Only one class present in y_true. ROC AUC is not defined, setting to 0.0.")
        roc_auc = 0.0
    
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        'kappa': cohen_kappa_score(y_true, y_pred),
        'f1': f1,
        'mcc': mcc,
        'logloss': logloss,
        'roc_auc': roc_auc,
        'pr_auc': avg_prec
    }
    return metrics

def find_optimal_threshold(y_true, y_proba, metric_func=f1_score, pos_label=1):
    """Находит оптимальный порог бинаризации по заданной метрике."""
    thresholds = np.linspace(0.01, 0.99, 100)
    best_score = -1
    best_threshold = 0.5
    
    if y_proba.ndim > 1 and y_proba.shape[1] == 2:
        y_proba = y_proba[:, 1]
    
    y_true = np.asarray(y_true)
    
    for threshold in thresholds:
        y_pred_temp = (y_proba >= threshold).astype(int)
        score = metric_func(y_true, y_pred_temp, pos_label=pos_label, zero_division=0)
        if score > best_score:
            best_score = score
            best_threshold = threshold
    return best_threshold

# ========= 5. Загрузка и Подготовка Данных =========
print(f"Версия scikit-learn: {sklearn.__version__}")
print(f"Загрузка данных из: {DATA_PATH}")
df_processed, categorical_features, numeric_features = load_and_preprocess_data(DATA_PATH)

if df_processed.empty:
    raise ValueError("Не удалось загрузить или обработать данные.")

print(f"Фильтрация данных для локации: {LOCATION_TO_ANALYZE}")
if LOCATION_TO_ANALYZE == 'Other':
    df_loc = df_processed[df_processed['Location_General'] != 'COR']
elif LOCATION_TO_ANALYZE == 'COR':
    df_loc = df_processed[df_processed['Location_General'] == 'COR']
else:
    df_loc = df_processed.copy()

if df_loc.empty:
    raise ValueError(f"Нет данных для выбранной локации: {LOCATION_TO_ANALYZE}")

print(f"Размер выборки после фильтрации: {df_loc.shape[0]}")

if 'sign_ddG' in df_loc.columns:
    print(f"Распределение классов:\n{df_loc['sign_ddG'].value_counts(normalize=True)}")
    target_col = 'sign_ddG'
else:
    raise ValueError("Целевая колонка 'sign_ddG' не найдена в отфильтрованных данных.")

X = df_loc[categorical_features + numeric_features]
y = df_loc[target_col]

print(f"NaN в X перед split: {X.isna().sum().sum()}")
print(f"NaN в y перед split: {y.isna().sum().sum()}")

if X.isna().sum().sum() > 0 or y.isna().sum().sum() > 0:
    print("Warning: Обнаружены NaN перед разделением, хотя они должны были быть удалены. Повторная очистка...")
    df_loc = df_loc.dropna(subset=categorical_features + numeric_features + [target_col])
    X = df_loc[categorical_features + numeric_features]
    y = df_loc[target_col]
    print(f"Новый размер выборки после очистки: {df_loc.shape[0]}")

print(f"Разделение данных: test_size={TEST_SIZE}, random_state={RANDOM_STATE}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,
                                                    random_state=RANDOM_STATE, stratify=y)
print(f"Размер обучающей выборки: {X_train.shape}, тестовой: {X_test.shape}")
print(f"Распределение классов в train:\n{y_train.value_counts(normalize=True)}")
print(f"Распределение классов в test:\n{y_test.value_counts(normalize=True)}")

# ========= 6. Препроцессор =========
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'), categorical_features),
        ('num', StandardScaler(), numeric_features)
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
)

try:
    preprocessor.set_output(transform='pandas')
    PREPROCESSOR_OUTPUT_PANDAS = True
    print("Препроцессор будет выводить Pandas DataFrame.")
except AttributeError:
    PREPROCESSOR_OUTPUT_PANDAS = False
    print("Warning: preprocessor.set_output не доступен. Вывод будет NumPy array.")

# ========= 7. Лучшие Модели с Гиперпараметрической Оптимизацией =========
rf_params = {
    'n_estimators': IntDistribution(100, 500),
    'max_depth': IntDistribution(3, 20),
    'min_samples_split': IntDistribution(2, 11),
    'class_weight': CategoricalDistribution(['balanced', None])
}

gb_params = {
    'n_estimators': IntDistribution(100, 500),
    'learning_rate': FloatDistribution(0.01, 0.3),
    'max_depth': IntDistribution(3, 15),
    'subsample': FloatDistribution(0.6, 1.0)
}

svc_params = {
    'C': FloatDistribution(0.1, 100),
    'kernel': CategoricalDistribution(['rbf', 'poly']),
    'gamma': CategoricalDistribution(['scale', 'auto'])
}

opt_rf = OptunaSearchCV(
    estimator=RandomForestClassifier(random_state=RANDOM_STATE),
    param_distributions=rf_params,
    cv=StratifiedKFold(n_splits=CV_FOLDS),
    n_trials=30,
    scoring='f1',
    n_jobs=-1
)

opt_gb = OptunaSearchCV(
    estimator=GradientBoostingClassifier(random_state=RANDOM_STATE),
    param_distributions=gb_params,
    cv=StratifiedKFold(n_splits=CV_FOLDS),
    n_trials=30,
    scoring='f1',
    n_jobs=-1
)

# ========= 8. Ансамблевая Модель =========
stacking_model = StackingClassifier(
    estimators=[
        ('rf', opt_rf),
        ('gb', opt_gb),
        ('lr', LogisticRegression(max_iter=1000))
    ],
    final_estimator=LogisticRegression(),
    cv=StratifiedKFold(n_splits=CV_FOLDS)
)

# ========= 9. Пайплайн с Ресэмплингом =========
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', SelectKBest(score_func=f_classif, k=N_FEATURES_TO_SELECT)),
    ('resampler', SMOTE(random_state=RANDOM_STATE)),
    ('classifier', stacking_model)
])

# ========= 10. Обучение и Оценка =========
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

metrics = calculate_metrics(y_test, y_pred, y_proba)

# ========= 11. Интерпретация =========
explainer = shap.Explainer(pipeline.named_steps['classifier'])
shap_values = explainer.shap_values(pipeline.named_steps['preprocessor'].transform(X_test))

plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values, pipeline.named_steps['preprocessor'].transform(X_test))
plt.tight_layout()
plt.savefig('shap_summary_2.png')

# ========= 12. Сохранение Результатов =========
results = {
    'best_params': {
        'rf': opt_rf.best_params_,
        'gb': opt_gb.best_params_
    },
    'metrics': metrics,
    'feature_importance': dict(zip(
        pipeline.named_steps['preprocessor'].get_feature_names_out(),
        pipeline.named_steps['classifier'].feature_importances_
    ))
}

with open('enhanced_model_results_2.json', 'w') as f:
    json.dump(results, f, indent=4)

import joblib
joblib.dump(pipeline, 'enhanced_model_2.joblib')

print("Расширенная модель успешно обучена и сохранена!")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=25296fc4-2e1a-40e5-990d-2e3a019f757c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>